In [35]:
import pandas as pd
import numpy as np

from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.datasets import load_digits

from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier 

from tqdm import tqdm

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats.distributions import randint

In [4]:
dataset = load_digits()
X, y = dataset['data'], dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

#### Задание 6.6.1


В скринкасте мы разобрали схему генерации признаков в стекинге, когда для тестовой выборки алгоритм заново переобучался на всей тренировочной выборке. Реализуйте схему, когда вместо этого производится агрегация ответов всех обученных на фолдах классификаторов на тестовой выборке при помощи усреднения.
Логика решения:

1) Создадим X_meta_test, заполним его нулями (по аналогии с X_meta_train);

2) Далее на каждом шаге, где мы обучаем folded_clf.fit (X_fold_train, y_fold_train) и его предсказания на X_fold_predict запихиваем в X_meta_train[predict_fold_index], добавим ещё одну строку, где в X_meta_test будем добавлять предсказания вероятностей folded_clf на X_test. Их можно сразу складывать друг с другом или сохранить много массивов, тогда в конце их нужно будет все сложить, а потом делить на количество сплитов (количество массивов равно количеству сплитов в кросс-валидации);

3) После цикла останется только усреднить все эти массивы — это и будет наш X_meta_test.

За основу нужно взять следующий код:

In [5]:
def compute_meta_feature_mean(clf, X_train, X_test, y_train, cv):
    """    Эта функция подсчитывает признаки для мета-классификатора.     Они являются вероятностями классов при решении задачи многоклассовой классификации.    :arg clf: классификатор    :args X_train, y_train: обучающая выборка    :arg X_test: признаки тестовой выборки    :arg cv: класс, генерирующий фолды (KFold)    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок    """
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)

    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)

    X_meta_test = meta_clf.predict_proba(X_test)

    return X_meta_train, X_meta_test

In [10]:
def compute_meta_feature_mean(clf, X_train, X_test, y_train, cv):
    """
    Эта функция подсчитывает признаки для мета-классификатора. 
    Они являются вероятностями классов при решении задачи многоклассовой классификации.

    :arg clf: классификатор
    :args X_train, y_train: обучающая выборка
    :arg X_test: признаки тестовой выборки
    :arg cv: класс, генерирующий фолды (KFold)

    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок
    """
# Напишите ваш код ниже   
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32)
    X_meta_test = 0
    i = 0
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
        X_meta_test = X_meta_test + folded_clf.predict_proba(X_test)
        i = i+1

    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)

    X_meta_test = X_meta_test/i
    
    return X_meta_train, X_meta_test 

code above is correct

In [12]:
#Решение от автора:
def compute_meta_feature_mean(clf, X_train, X_test, y_train, cv):
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32)
    X_meta_test = np.zeros((len(X_test), n_classes), dtype=np.float32)
    
    for train_fold_index, predict_fold_index in cv.split(X_train):        
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
        
        # We sum all of the predictions across folds and then divide the sum by the number of folds
        X_meta_test += folded_clf.predict_proba(X_test)
    
    X_meta_test /= cv.n_splits
    return X_meta_train, X_meta_test   
         

Для следующих заданий используйте этот код:

In [14]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test

In [15]:
def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
   
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])
    
    return stacked_features_train, stacked_features_test

In [16]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

def compute_metric(clf, X_train=X_train, y_train=y_train, X_test=X_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

#### Задание 6.6.2

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

- логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимое количество итераций — 2000
    
- логистическая регрессия с L2-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — multinomial, максимальное допустимое количество итераций — 2000
    
- случайный лес из 300 деревьев
    
- градиентный бустинг из 200 деревьев

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [43]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(C=0.001, penalty='l1', solver='saga', max_iter=2000),
    LogisticRegression(C=0.001, penalty='l2', solver='saga', max_iter=2000),  
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    GradientBoostingClassifier(n_estimators=200)
], X_train, X_test, y_train, cv)



  0%|                                                     | 0/4 [00:00<?, ?it/s]

 25%|███████████▎                                 | 1/4 [00:38<01:56, 38.75s/it]

 50%|██████████████████████▌                      | 2/4 [00:40<00:55, 27.66s/it]

 75%|█████████████████████████████████▊           | 3/4 [00:52<00:22, 22.83s/it]

100%|█████████████████████████████████████████████| 4/4 [03:10<00:00, 57.47s/it]

In [41]:
np.random.seed(42)
clf = LogisticRegression(multi_class='auto', solver='lbfgs')
clf.fit(stacked_features_train, y_train)
accuracy_score(clf.predict(stacked_features_test), y_test)

0.9888888888888889

In [45]:
compute_metric(LogisticRegression(multi_class='auto', solver='lbfgs'), X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.986327

    0.9833333333333333 is correct
    0.986327 is also correct

#### Задание 6.6.3

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

- метод ближайшего соседа (k-NN) со стандартными параметрами; # https://proglib.io/p/metod-k-blizhayshih-sosedey-k-nearest-neighbour-2021-07-19
- случайный лес из 300 экстремальных деревьев.

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [48]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features([
    KNeighborsClassifier(), 
    RandomForestClassifier(n_estimators=300, n_jobs=-1)
], X_train, X_test, y_train, cv)



  0%|                                                     | 0/2 [00:00<?, ?it/s]

 50%|██████████████████████▌                      | 1/2 [00:00<00:00,  7.94it/s]

100%|█████████████████████████████████████████████| 2/2 [00:11<00:00,  3.51s/it]

In [36]:
np.random.seed(42)
clf = LogisticRegression(multi_class='auto', solver='lbfgs')
clf.fit(stacked_features_train, y_train)
accuracy_score(clf.predict(stacked_features_test), y_test)

0.9861111111111112

In [49]:
compute_metric(LogisticRegression(multi_class='auto', solver='lbfgs'), X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.990099

    0.9861111111111112 is correct
    0.990099 is also correct

#### Задание 6.6.4

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

- логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимально допустимое количество итераций — 2000;
- метод ближайшего соседа со стандартными параметрами;
- случайный лес из 300 экстремальных деревьев;
- AdaBoost со стандартными параметрами.

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [50]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(C=0.001, penalty='l1', solver='saga', max_iter=2000),
    KNeighborsClassifier(),  
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    AdaBoostClassifier()
], X_train, X_test, y_train, cv)



  0%|                                                     | 0/4 [00:00<?, ?it/s]

 25%|███████████▎                                 | 1/4 [00:39<01:57, 39.29s/it]

 50%|██████████████████████▌                      | 2/4 [00:39<00:55, 27.54s/it]

 75%|█████████████████████████████████▊           | 3/4 [00:51<00:22, 22.99s/it]

100%|█████████████████████████████████████████████| 4/4 [00:54<00:00, 17.01s/it]

In [39]:
np.random.seed(42)
clf = LogisticRegression(multi_class='auto', solver='lbfgs')
clf.fit(stacked_features_train, y_train)
compute_metric(clf.predict(stacked_features_test), y_test)

0.9888888888888889

In [51]:
compute_metric(LogisticRegression(multi_class='auto', solver='lbfgs'), X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.987798

    0.9888888888888889 is correct
    0.987798 is also correct

#### Задание 6.6.5

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

- случайный лес из 300 деревьев;
- случайный лес из 300 экстремальных деревьев.

Для генерации фолдов используйте класс StratifiedKFold, который позволяет делать так называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов).
Для корректной работы необходимо подправить код в функции compute_meta_feature. Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric. Количество фолдов = 10

In [52]:
from sklearn.model_selection import StratifiedKFold

In [78]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    # skf = StratifiedKFold(n_splits=10)
    # skf.split(X_train, y_train)    
    splits = cv.split(X_train, y_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test

In [79]:
cv = StratifiedKFold(n_splits=10)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1), # n_jobs=-1 means using all processors. 
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1)
], X_train, X_test, y_train, cv)





  0%|                                                     | 0/2 [00:00<?, ?it/s]



 50%|██████████████████████▌                      | 1/2 [00:12<00:12, 12.59s/it]



100%|█████████████████████████████████████████████| 2/2 [00:21<00:00, 11.62s/it]

In [80]:
compute_metric(LogisticRegression(multi_class='auto', solver='lbfgs'), X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.984745

0.981296 is correct

In [36]:
np.random.seed(42)
clf = LogisticRegression(multi_class='auto', solver='lbfgs')
clf.fit(stacked_features_train, y_train)
accuracy_score(clf.predict(stacked_features_test), y_test)

0.9861111111111112

0.9861111111111112 is also correct

#### Задание 6.6.6

В задании 6.6.5 измените 10 фолдов на 20. Укажите полученное качество.

In [73]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    # skf = StratifiedKFold(n_splits=20)
    # skf.split(X_train, y_train)
    splits = cv.split(X_train, y_train)
    
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test

In [74]:
cv = StratifiedKFold(n_splits=20)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1), # n_jobs=-1 means using all processors. 
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1)
], X_train, X_test, y_train, cv)





  0%|                                                     | 0/2 [00:00<?, ?it/s]



 50%|██████████████████████▌                      | 1/2 [00:22<00:22, 22.29s/it]



100%|█████████████████████████████████████████████| 2/2 [00:40<00:00, 20.92s/it]

In [75]:
compute_metric(LogisticRegression(multi_class='auto', solver='lbfgs'), X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.983602

    0.980988 is correct
    0.983602 is alco correct

#### Задание 6.6.7

В задании 6.6.6 укажите количество фолдов равным 5 и поменяйте мета-алгоритм на случайный лес со стандартными параметрами. Укажите полученное качество.

In [76]:
cv = StratifiedKFold(n_splits=5)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1), # n_jobs=-1 means using all processors. 
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1)
], X_train, X_test, y_train, cv)





  0%|                                                     | 0/2 [00:00<?, ?it/s]



 50%|██████████████████████▌                      | 1/2 [00:06<00:06,  6.24s/it]



100%|█████████████████████████████████████████████| 2/2 [00:11<00:00,  5.95s/it]

In [77]:
compute_metric(RandomForestClassifier(), X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.985118

0.985118 is correct

#### Задание 6.6.8

В задании 6.6.5 поменяйте мета-алгоритм на метод ближайших соседей (k-NN) со стандартными параметрами. Укажите полученное качество.

In [83]:
cv = StratifiedKFold(n_splits=10)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1), # n_jobs=-1 means using all processors. 
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1)
], X_train, X_test, y_train, cv)





  0%|                                                     | 0/2 [00:00<?, ?it/s]



 50%|██████████████████████▌                      | 1/2 [00:11<00:11, 11.61s/it]



100%|█████████████████████████████████████████████| 2/2 [00:21<00:00, 11.07s/it]

In [84]:
compute_metric(KNeighborsClassifier(), X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.98762

0.98762 is correct

#### Задание 6.6.9

В задании 6.6.6 поменяйте мета-алгоритм на градиентный бустинг со стандартными параметрами. Укажите полученное качество.

In [91]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) 

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42), # n_jobs=-1 means using all processors. 
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=42)
], X_train, X_test, y_train, cv)





  0%|                                                     | 0/2 [00:00<?, ?it/s]



 50%|██████████████████████▌                      | 1/2 [00:13<00:13, 13.63s/it]



100%|█████████████████████████████████████████████| 2/2 [00:22<00:00, 12.33s/it]

In [92]:
clf = GradientBoostingClassifier(random_state=42)
compute_metric(clf, X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.981895

    0.984554 is correct
    0.986593 is also correct
    0.989765 is also correct
    0.981895 is also correct

#### Задание 6.6.10

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

- случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24;
- случайный лес из 300 экстремальных деревьев.

Для генерации фолдов используйте класс StratifiedKFold, который позволяет делать так называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов).
Для генерации фолдов используйте класс StratifiedKFold и исправленный вами ранее код в функции compute_meta_feature.
Выполните разбиение на 3 фолда.
Как мета-алгортм используйте случайный лес из 100 экстремальных деревьев. Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [93]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42) 

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestClassifier(n_estimators=300, criterion='gini', max_depth=24, n_jobs=-1, random_state=42), # n_jobs=-1 means using all processors. 
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=42)
], X_train, X_test, y_train, cv)





  0%|                                                     | 0/2 [00:00<?, ?it/s]



 50%|██████████████████████▌                      | 1/2 [00:04<00:04,  4.46s/it]



100%|█████████████████████████████████████████████| 2/2 [00:07<00:00,  4.10s/it]

In [94]:
clf = ExtraTreesClassifier(n_estimators=100, n_jobs=-1, random_state=42)
compute_metric(clf, X_train=stacked_features_train, y_train=y_train, X_test=stacked_features_test)

0.986498

0.986498 is correct

#### Задание 6.6.11

Обучите на тренировочной выборке следующие алгоритмы:

- случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24;
- случайный лес из 300 экстремальных деревьев;
- логистическую регрессию со стандартными параметрами.

Усредните их ответы на тестовой выборке методом сложения предсказаний и затем взятия функции argmax: answer = (prediction1 + prediction2 + prediction3).argmax(axis = 1). Посчитайте качество, аналогично функции compute_metric (F1-score с макро-усреднением, округленный до 6 знака).

expert: 
нам нужно усреднить ответы алгоритмов. Усреднением здесь мы добиваемся большей точности предсказаний.
Однако усреднять здесь (в случае задачи классификации) нужно не метки классов, возвращаемые методом predict() (что некорректно: допустим, три наши алгоритма дали такие предсказания меток классов: 1, 1 и 7 - усреднение меток здесь даёт ответ 3, что противоречит здравому смыслу).
"Усреднять" нужно "сырые" ответы - вероятности принадлежности классов, возвращаемые методом predict_proba()  : вернёт таблицу с числом столбцов равным числу классов - в нашем случае 10. По столбцам будут расположены вероятности принадлежности объекта к классам.Это делается следующим образом: для каждого объекта мы складываем вероятности, выданные алгоритмами и делим их на количество (усредняем). В качестве предсказания мы берём класс, для которого получившаяся вероятность максимальна.В коде это может выглядеть так:

In [105]:
def compute_predicts(classifiers, X_train, y_train, X_test):
    predicts = []
    for clf in classifiers:
        clf.fit(X_train, y_train)
        predicts.append(clf.predict_proba(X_test))
    return predicts




predicts = compute_predicts([ 
    RandomForestClassifier(n_estimators=300, criterion='gini', max_depth=24, n_jobs=-1, random_state=42),
    ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=42), 
    LogisticRegression()
], X_train, y_train, X_test)

y_test_pred = np.sum(predicts, axis=0).argmax(axis=1) #argmax() как раз и используется, чтобы получить индекс столбца, для которого вероятность максимальна.

#P.S. я использую np.sum()  вместо np.mean() , но тут разницы нет, поскольку мне нужен лишь _индекс_ максимального элемента в строке, 
#а он не измениться, если весь массив умножить/разделить на какое-то (положительное) число

print(np.round(f1_score(y_test, y_test_pred, average='macro'), 6))

0.976259


/home/dex/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


my version below:

In [102]:
rfc = RandomForestClassifier(n_estimators=300, criterion='gini', max_depth=24, n_jobs=-1, random_state=42)
rfc.fit(X_train, y_train)
y_pred_1 = rfc.predict_proba(X_test)

etc = ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=42)
etc.fit(X_train, y_train)
y_pred_2 = etc.predict_proba(X_test)

lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)
y_pred_3 = lr.predict_proba(X_test)


/home/dex/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [104]:
answer = (y_pred_1 + y_pred_2 + y_pred_3).argmax(axis = 1)
np.round(f1_score(y_test, answer, average='macro'), 6)

0.976259

0.976259 is correct